In [1]:
import csv
from datetime import datetime, timedelta
import os
import pprint

import ee
import pandas as pd


In [2]:
ee.Initialize(
    project='ee-cmorton',
    opt_url='https://earthengine-highvolume.googleapis.com'
)

In [3]:
et_cell_path = 'et_cell_metadata.csv'
# et_cell_path = os.path.join(os.getcwd(), 'et_cell_metadata.csv')

eto_ws = os.path.join(os.getcwd(), 'eto')
eto_file_prefix = 'eto'

years = list(range(1985, 2024))
overwrite_flag = False


## Read in the list of target grid cells/points to process

In [4]:
et_cell_df = pd.read_csv(et_cell_path)
print(f'ET cell count: {len(et_cell_df)}')


ET cell count: 11


## Extract and save the ETo time series

In [5]:
# We will probably not use all the years, but build all the folder
for year in years:
    if not os.path.isdir(os.path.join(eto_ws, f'{year}')):
        os.makedirs(os.path.join(eto_ws, f'{year}'))

eto_files = {
    item for year in years
    for item in os.listdir(os.path.join(eto_ws, f'{year}')) 
    if item.endswith('.csv')
}
print(f'ETo files: {len(eto_files)}\n')
#pprint.pprint(eto_files)


for i, et_cell in et_cell_df.iterrows():
    cell_id = et_cell["GRIDMET_ID"]
    xy = [et_cell['LON'], et_cell['LAT']]
    print(f'{i:>4d} - Cell {et_cell["GRIDMET_ID"]}')
    
    for year in years:
        file_name = f'{eto_file_prefix}_{year}_{cell_id}.csv'
        file_path = os.path.join(eto_ws, f'{year}', file_name)
        if not overwrite_flag and (file_name in eto_files):
            continue
        # print(f'  {year}')
    
        # Get the reference ET timeseries
        daily_info = (
            ee.ImageCollection('projects/openet/assets/reference_et/conus/gridmet/daily/v1')
            .filterDate(f'{year}-01-01', f'{year+1}-01-01')
            .select(['eto'])
            .getRegion(ee.Geometry.Point(xy), 4000)
            .getInfo()
        )

        # Reformat the getInfo output so it can be easily read into a dataframe
        daily_dict = {'date': {}, 'eto': {}}
        for row in daily_info[1:]:
            date = datetime.utcfromtimestamp(row[3] / 1000.0).strftime('%Y-%m-%d')
            daily_dict['date'][date] = date
            daily_dict['eto'][date] = row[4]
            
        # Build and save the daily dataframe
        daily_df = pd.DataFrame.from_dict(daily_dict)
        #daily_df['cell_id'] = et_cell['cell_id']
        daily_df['date'] = pd.to_datetime(daily_df['date'], format='%Y-%m-%d')
        daily_df.sort_index(inplace=True)
        daily_df.to_csv(file_path, index=False)

        #del daily_df, daily_dict, daily_info

print('\nDone')


ETo files: 429

   0 - Cell 387098
   1 - Cell 467476
   2 - Cell 473008
   3 - Cell 511799
   4 - Cell 581084
   5 - Cell 605776
   6 - Cell 661167
   7 - Cell 686149
   8 - Cell 688961
   9 - Cell 564161
  10 - Cell 565566

Done
